In [7]:
# Importing this just for fun!
import this
import twitter 
# Import json package
import json
# Import sqlite3 package
import sqlite3

# Import re package
import re
# Import time package
import time
# Import sys package
import sys

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

ImportError: No module named twitter

In [ ]:
%cd "/Users/mtoasf/Downloads"

In [ ]:
consumer_key='7Rl2dNWbahxQcvHjrdqNo9Azc'
consumer_secret='FlksRioJcF8fDw5VtkYQ2DXgOfI1Yu27ngwqw9BvrLsShx2TfW'

access_token='3194888576-G35yt1YrrtSJ0FoHHDLLD5UMDKisQSWZYZixGKQ'
access_token_secret='Xq4Gd1qDoGe075ek3GSnM5zZuILYZY4t3IteXeQWh8hQp'

#auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
#auth.set_access_token(access_token,access_token_secret)

api=twitter.Api(consumer_key,consumer_secret,access_token,access_token_secret)

In [ ]:
def initiate_sqlite():

    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    
    # Adding "TWEETS" table to the database
    conn.execute('''CREATE TABLE TWEETS (ID TEXT PRIMARY KEY,
                 TIME DATE,
                 NAME TEXT, 
                 COORD1 REAL,
                 COORD2 REAL,
                 CONTENT TEXT);''')
    
    conn.commit()
    conn.close()
    
    return


In [ ]:
def sqlite_write(twitter_id, screen_name_of_user, coord1, coord2, text_of_tweet):
    
    # Connecting to "monitoring" database already set up before using sqlite3
    conn = sqlite3.connect("monitoring.db")
    
    # Insert Query "TWEETS" table and excecuting the command
    # using a sqlite timestamp for the time column
    command = "INSERT INTO TWEETS VALUES ('"+twitter_id+"',datetime('now', 'localtime'),'"+screen_name_of_user+"','"+str(coord1)+"','"+str(coord2)+"', '"+text_of_tweet+"');"
    conn.execute(command)
    
    # Commiting and closing the connection to the database
    conn.commit()
    conn.close()
    
    return

In [ ]:
def sqlite_read():
    
    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    c = conn.cursor()
    
    # Select Query "TWEETS" table for the tweets added to the database in the last 10 minutes
    command = "SELECT * FROM TWEETS WHERE TIME BETWEEN datetime('now', 'localtime', '-10 minutes') AND datetime('now', 'localtime');"
    c.execute(command)
    
    # fetching the selected data into the rows list
    rows = c.fetchall()
    
    # closing the connection to the database
    conn.close()
    
    return rows

In [ ]:
def sqlite_count():
    
    # Connecting to "monitoring" database already set before up using sqlite3
    conn = sqlite3.connect("monitoring.db")
    c = conn.cursor()
    
    # Select Query "TWEETS" table for the tweets added to the database in the last 10 minutes
    
    command = "SELECT COUNT(*) FROM TWEETS WHERE TIME BETWEEN datetime('now', 'localtime', '-1000 minutes') AND datetime('now', 'localtime');"
    c.execute(command)
    
    # fetching the count data
    count = c.fetchall()
    
    # closing the connection to the database
    conn.close()
    
    return count

class StdoutListener(StreamListener):
    def on_data(self,data):
        try:
            print (data)
            savefile=open('D:/SIT_Class/EE 551 Python/crawler/twitter333.txt','a')
            savefile.write(data)
            savefile.write("\n")
            savefile.close()
            return True
        except BaseException as e:
            print("Failed on Data",str(e))
            time.sleep(5)
    def on_erro(self,status):
        print(status)

In [ ]:
def start_stream():
    
    # Define Hashtag+Keywords and languages for streaming API filter
    
    # e.g. ["#Berlin terror"] would be to filter for "#Berlin" AND "terror"
    # e.g. ["#Berlin terror, #Berlin terroranschlag, #Berlin anschlag"]
    # for easier testing we leave this with "#Berlin" only
    hashtag_to_monitor = ["#Berlin"]
    
    # filtering for english and german language tweets only
    LANGUAGES = ["en", "de"]
    
    # Connecting to stream
    stream = api.GetStreamFilter(track=hashtag_to_monitor, languages=LANGUAGES)        
        
    # Going through the tweets delivered and saving the selected data to the database
    try:
        for tweet in stream:
            
            # make the tweet text suitable for saving in the database and sending by email
            # e.g. removing links, having just text as output for easy readability
            # and understanding. this also helps as a workaround to unsolved problems
            # to send unicode messages by smtplib in python3
            tweet["text"] = re.sub("\'", "", tweet["text"])
            tweet["text"] = re.sub("\"", "\\\"", tweet["text"])
            tweet["text"] = re.sub("\s", " ", tweet["text"])
            tweet["text"] = re.sub("http\S+", "", tweet["text"])
            tweet["text"] = re.sub("\W", " ", tweet["text"])
            
            # calling the sqlite_write function and saving the data to the database
            
            if tweet["coordinates"] != None:
            
                sqlite_write(tweet["id_str"], tweet["user"]["screen_name"], tweet["coordinates"]["coordinates"][0], tweet["coordinates"]["coordinates"][1],tweet["text"])
            else:
                sqlite_write(tweet["id_str"], tweet["user"]["screen_name"], "NULL", "NULL", tweet["text"])
            
            # calling threshold function to check if an alert has to be sent
            #threshold()
    except Exception as g:
        print("Exception", g)
        
    return

In [ ]:
initiate_sqlite()

In [ ]:
start_stream()